In [30]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from langdetect import detect
import re

# Mapeo de códigos de idioma de langdetect a spacy
LANGUAGE_MAP = {
    'en': 'en_core_web_md',
    'es': 'es_core_news_md',
    'fr': 'fr_core_news_md',
    'de': 'de_core_news_md'
    # Agrega más mapeos si es necesario
}

# Cargar modelos de spaCy según el idioma
def cargar_modelo_spacy(idioma):
    try:
        return spacy.load(LANGUAGE_MAP[idioma])
    except KeyError:
        raise ValueError(f'Modelo de spaCy no disponible para el idioma {idioma}')

# Detectar automáticamente el idioma del texto
def detectar_idioma(texto):
    return detect(texto)

# Cargar modelo BERT multilingüe
model = SentenceTransformer('distiluse-base-multilingual-cased')

def preprocesar_texto(texto):
    # Reemplazar saltos de línea por espacios
    texto = re.sub(r'\n+', ' ', texto)
    # Borrar los |
    texto = texto.replace('|', '.')
    # Normalizar comillas y >> o <<
    texto = texto.replace('“', '"').replace('”', '"')
    texto = texto.replace('»', '"')
    texto = texto.replace('«', '"')
    return texto

def extraer_frases_clave_avanzado(texto):
    # Preprocesar el texto para manejar saltos de línea y comillas
    texto = preprocesar_texto(texto)

    # Detectar el idioma del texto
    idioma_detectado = detectar_idioma(texto)
    nlp = cargar_modelo_spacy(idioma_detectado)
    
    # Procesar el texto con spaCy para obtener las oraciones
    doc = nlp(texto)
    oraciones = [sent.text.strip() for sent in doc.sents]
    
    # Representar las oraciones seleccionadas usando BERT
    oraciones_embed = model.encode(oraciones, convert_to_tensor=True)

    # Calcular similitudes entre las oraciones y seleccionar las más representativas
    similitudes = util.pytorch_cos_sim(oraciones_embed, oraciones_embed)

    # Filtrar frases que son similares para evitar redundancia
    frases_clave = []
    for idx, oracion in enumerate(oraciones):
        if not any(similitudes[idx][i] > 0.9 for i in range(idx)):
            frases_clave.append(oracion)

    # Retornar las frases clave completas
    return frases_clave

# Ejemplo de uso con texto en varios idiomas
texto_es = """Entre 150 y 200 migrantes han intentado acceder a nado a Ceuta desde Marruecos, la mayoría en la madrugada de este lunes, aunque son varios días consecutivos de intentos, según han informado a Efe fuentes de la Guardia Civil. Los agentes se han desplegado desde las primeras horas de la noche del domingo y lograron interceptar a unas 50 personas que se habían lanzado al mar con el objetivo de entrar en la ciudad autónoma. Agentes de ambos territorios han colaborado para interceptar a las personas, entre las que también había menores. Según se les ha alcanzado, se les ha devuelto a Marruecos, aunque fuentes de las fuerzas de seguridad han contado a EL PAÍS que sobre el medio día aún había unas 54 personas pendientes de devolución a un ritmo más lento. Se desconoce todavía el motivo que ha impulsado a tanta gente a intentar el cruce, aunque se señala, como en otras ocasiones, a la niebla que dificulta visibilidad de los agentes y a cierta laxitud de las autoridades marroquíes.

La actividad ha sido intensa durante toda la madrugada. Los medios locales han descrito los gritos que no han dejado de oírse en la oscuridad del mar. “Los gritos se escuchaban desde la carretera. Gritos desgarradores de gente que estaba en el agua y no aguantaba más. El Servicio Marítimo acudía hacia esos puntos buscando dar con quienes pedían ayuda, mientras que agentes con cámaras térmicas daban las ubicaciones de los puntos de calor”, ha descrito El Faro de Ceuta.

Una patrullera de la Guardia Civil interviene durante una intensa noche en la frontera del Tarajal en Ceuta. 
Una patrullera de la Guardia Civil interviene durante una intensa noche en la frontera del Tarajal en Ceuta.
Joaquín Sánchez
La operación, en consecuencia, se ha mantenido gran parte de la noche, con las patrullas de la Guardia Civil yendo y viniendo de un lado a otro de la costa. Mientras el Servicio Marítimo del Instituto Armado, además de Salvamento Marítimo, permanecieron en el mar durante horas intentando localizar y recogiendo a los nadadores. Durante la madrugada también intervino Cruz Roja atendiendo a los migrantes que lograban poner un pie en la playa con cuadros de hipotermia.

La comandancia se resiste a cifrar el número de personas que han conseguido entrar en Ceuta y que iban de camino a centros de menores o al Centro de Estancia Temporal de Inmigrantes (CETI), que ya está por encima de su capacidad. Están en marcha las labores de identificación por parte de la Policía Nacional, mientras el Gobierno de la ciudad autónoma aguarda inquieto a conocer el número exacto de menores que se encuentran en el grupo, un colectivo al que debe atender con sus instalaciones desbordadas con unos 400 niños y adolescentes. Solo esta madrugada han sido al menos siete, un número que asciende a 47 si se contabiliza todo el fin de semana.

La semana pasada también estuvo marcada por intentos de entrada frustrados y accesos a Ceuta desde Marruecos, en su mayoría a nado (aunque contabilizados como vía terrestre por el Gobierno). Una veintena de varones argelinos y marroquíes, nueve de ellos menores, lograron entrar en la ciudad autónoma durante la madrugada del 4 de agosto, mientras en la noche siguiente lo consiguieron otros dos adultos procedentes de Argelia, según confirmó la Guardia Civil.

"""
texto_en = """THE SOLDIERS chanted the Lord’s Prayer and clicked rosary beads as they advanced. For Ivan, 43, an old-timer from Ukraine’s 103rd brigade, the fighting inside Russia was just another day’s work. “Grenades and mortars look the same wherever you are.” The newest recruits were almost paralysed with fear. But the men tramped on together, 10km a day, crossing fields and railway lines, every night replacing forward units in hastily dug positions ahead of them. Three days, three hikes, three rotations. On the third night, the Russian glide bombs hit. “Everything was burning. Arms here, legs there”. Twelve men in the company died immediately. Ivan suffered shrapnel injuries to his groin and chest, and was evacuated to a hospital in the Sumy region of Ukraine.

Ukraine’s six-day-long operation inside Russia has progressed faster than many dared believe. A Ukrainian security source says that by Saturday August 10th, some units had moved a full 40km inside Russia towards the regional capital of Kursk. The attack, shrouded in secrecy, caught the Kremlin off-guard. Some 76,000 locals have fled and the Russian authorities have declared a state of emergency there. The absence of a well-organised evacuation has angered many. Vladimir Putin called it a large-scale “provocation”. Volodymyr Artiukh, the head of Ukraine’s military administration in Sumy, says the Ukrainian success represented a “cold shower” for the Russians. “They are feeling what we have been feeling for years, since 2014. This is a historical event.”

Read more of our recent coverage of the Ukraine war

But the accounts from Ukraine’s wounded suggest it has not been a walk in the park, and remains risky. The hospital ward reeks of the sacrifice: soil, blood, and stale sweat. Foil burn-dressings line the corridor. In the yard, the patients, some wrapped like mummies from head to toe in bandages, smoke furiously. Angol, a 28-year-old paratrooper with the 33rd brigade, looks like a Christmas tree. His left arm is immobilised in a fixation device. Tubes, bags and wires protrude from his body. He was also about 30km into Russia when his luck ran out. He isn’t sure if it was artillery or a bomb that hit him. Maybe it was friendly fire; there was a lot of that. All he can remember is falling to the ground and shouting “300”, the code for wounded. The Russians had been on the run up to then, he insists, abandoning equipment and ammunition as fast as they could.


Map: The Economist
Other soldiers in the yard recall the demonic buzz of Russia’s skies. Ukraine has deployed a lot of air-defence and electronic-warfare assets to the area, but drones and aviation find ways through. Mykola, an infantryman who says he was in the first group to cross over into Russia, says pilots attacked as soon as they entered the first Russian village. At a second village, the group was targeted by helicopters. Mykola recalls throwing himself to the ground, and then the sound of a helicopter crashing, downed by a Ukrainian anti-aircraft missile. But close calls have consequences. The problem with throwing yourself to the ground at night is you can’t see where you’re falling, Mykola says. He broke a rib and had to be evacuated.

Some aspects of Ukraine’s operation appear to have been meticulously planned. Operational security delivered the element of surprise, a crucial aspect of warfare. “We sent our most combat-ready units to the weakest point on their border,” says a general-staff source deployed to the region. “Conscript soldiers faced paratroopers and simply surrendered.” But other aspects of the operation indicate a certain haste in preparation. All three soldiers quoted in this article were pulled, unrested, from under-pressure front lines in the east with barely a day’s notice.

The end goal of Ukraine’s operation still remains unclear: does it aim to push further, towards the city of Kursk? Is the plan to occupy part of the territory permanently, perhaps as a bargaining chip in negotiations, or does it intend to withdraw after causing Vladimir Putin maximum embarrassment? Ukraine does not appear to be reinforcing its positions in any serious sense. “Our calf demands a wolf,” the security source cautions, using a local saying to warn against overly ambitious objectives.

A minimum objective appears to be pulling troops away from Russia’s stranglehold in Kharkiv and Donbas, the main focuses of the war. On early evidence, the results are inconclusive. Russia has shifted troops from the Kharkiv front, but so far it has moved far fewer from the vital Donbas front. “Their commanders aren’t idiots,” says the Ukrainian general-staff source. “They are moving forces, but not as quickly as we would like. They know we can’t extend logistics 80 or 100 km.”

The source cautions against comparing the Kursk incursion to Ukraine’s successful swift recapture of much of Kharkiv province in late 2022. The Russian army is taking the war more seriously now, he says: “The danger is we’ll fall into a trap, and Russia will grind our teeth down.” On Sunday Russia’s defence ministry claimed, albeit not for the first time, that it had “thwarted” attempts by Ukrainian forces to break deeper into Russia.

The mathematics of war have never favoured Ukraine, which must husband its limited resources, and an assault deep inside undefended Russian territory risks making the situation worse. But the operation has already improved the one crucial intangible—morale—that has allowed Ukraine to cheat the odds for nearly three years now. Whether in government offices in Kyiv, or in front-line hospitals treating the wounded, the nation believes it has uncovered a vulnerability in Vladimir Putin’s armour. Tired, dirty and exhausted, the soldiers say they regret no part of the risky operation that has already killed scores of their comrades: they would rejoin it in a heartbeat. “For the first time in a long time we have movement,” says Angol. “I felt like a tiger.”"""
texto_fr = """Vous pouvez partager un article en cliquant sur les icônes de partage en haut à droite de celui-ci. 
La reproduction totale ou partielle d’un article, sans l’autorisation écrite et préalable du Monde, est strictement interdite. 
Pour plus d’informations, consultez nos conditions générales de vente. 
Pour toute demande d’autorisation, contactez syndication@lemonde.fr. 
En tant qu’abonné, vous pouvez offrir jusqu’à cinq articles par mois à l’un de vos proches grâce à la fonctionnalité « Offrir un article ». 

https://www.lemonde.fr/sport/article/2024/08/10/transports-publics-tirer-les-lecons-du-sans-faute-des-jo_6275186_3242.html

Des trains et des métros fréquents et à l’heure, des incidents rares et vite traités, un personnel en nombre pour informer, accompagner 4 millions d’usagers quotidiens… Les Franciliens n’ont pas reconnu leurs transports publics pendant les Jeux olympiques. A une période de dysfonctionnements multiples et récurrents alimentant le mécontentement ont miraculeusement succédé deux semaines qui ont déjoué tous les mauvais présages.

Lire aussi | Article réservé à nos abonnés Transports : une réussite olympique à pérenniser

Non seulement le pari audacieux d’organiser les premiers JO totalement accessibles en transports en commun décarbonés, à vélo ou à pied, a été gagné, non seulement la desserte des nombreux lieux de compétition disséminés dans Paris et dans sa région a été fluide, mais la preuve a été administrée de la capacité du réseau et de ses agents à assurer, grâce à des efforts programmés et à des moyens adaptés, des déplacements rapides, fiables et même agréables. Mieux encore, les Jeux de Paris ont donné le spectacle réjouissant d’une grande ville largement libérée, un temps, de l’emprise et des nuisances de la circulation automobile.

L’« exploit » dont se targue Valérie Pécresse, la présidente de la région Ile-de-France, chargée des transports, est si patent, si rare aussi, qu’il est impératif d’en tirer les leçons afin que cette « parenthèse enchantée » des JO de Paris ne vienne pas grossir le rayon des éphémères moments de grâce. Les millions d’usagers quotidiens des transports franciliens ont le droit de bénéficier du même traitement dans les transports que les spectateurs des Jeux.

Des recrutements pérennisés
Certes, l’héritage de « Paris 2024 » en matière de transports est important : outre le prolongement des lignes 14 du métro et E du RER, les Jeux ont été l’occasion de combler la pénurie de conducteurs post-Covid – des recrutements pérennisés –, d’améliorer la sécurité grâce à une meilleure coordination entre services de police et d’accélérer les procédures en cas d’incident. Quant au personnel, incité par des primes, il a fait la preuve d’une capacité de mobilisation remarquable, qui s’est notamment traduite par la disparition de l’absentéisme. Sans oublier la formidable vitrine pour un secteur exportateur que représente le sans-faute des transports publics français pendant les JO.

Mais les travaux interrompus vont reprendre dès le 12 août, accaparant des personnes affectées à la maintenance, et donc à la régularité, pendant les Jeux, les primes vont cesser et les « gilets violets » se disperser. La formidable réussite dont les élus vont se targuer n’a été possible que parce que des moyens exceptionnels ont été déployés. Et parce que, même avec le public des JO, le réseau reste moins sollicité l’été.

Lire aussi | Article réservé à nos abonnés Marcelle, agente de quai au gilet violet qui fluidifie les transports

Or, ces deux semaines de bus, métro et trains pratiques et ponctuels ne peuvent pas rester sans lendemain. Le prochain gouvernement doit confirmer et amplifier les investissements de l’Etat dans les transports publics. La promesse faite en février 2023 par la première ministre d’alors, Elisabeth Borne, de consacrer 100 milliards d’euros au ferroviaire d’ici à 2040, doit être réactivée, de même que le plan de multiplication des RER métropolitains ; l’accessibilité du métro parisien « d’ici à dix ans » aux personnes porteuses de handicap, que fait miroiter Mme Pécresse, doit être financée.

Les transports publics, symbole tangible du respect des pouvoirs publics pour les citoyens, constituent un sujet politique majeur. Les JO ont démontré la possibilité de réconcilier les Français avec eux. Tout doit être fait pour prolonger ce cercle vertueux et doter la France de transports de classe olympique."""

# Extraer frases clave
frases_clave_es = extraer_frases_clave_avanzado(texto_es)
frases_clave_en = extraer_frases_clave_avanzado(texto_en)
frases_clave_fr = extraer_frases_clave_avanzado(texto_fr)

print("Frases clave en español:")
for frase in frases_clave_es:
    print(frase)

print("\nFrases clave en inglés:")
for frase in frases_clave_en:
    print(frase)

print("\nFrases clave en francés:")
for frase in frases_clave_fr:
    print(frase)


c:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Frases clave en español:
Entre 150 y 200 migrantes han intentado acceder a nado a Ceuta desde Marruecos, la mayoría en la madrugada de este lunes, aunque son varios días consecutivos de intentos, según han informado a Efe fuentes de la Guardia Civil.
Los agentes se han desplegado desde las primeras horas de la noche del domingo y lograron interceptar a unas 50 personas que se habían lanzado al mar con el objetivo de entrar en la ciudad autónoma.
Agentes de ambos territorios han colaborado para interceptar a las personas, entre las que también había menores.
Según se les ha alcanzado, se les ha devuelto a Marruecos, aunque fuentes de las fuerzas de seguridad han contado a EL PAÍS que sobre el medio día aún había unas 54 personas pendientes de devolución a un ritmo más lento.
Se desconoce todavía el motivo que ha impulsado a tanta gente a intentar el cruce, aunque se señala, como en otras ocasiones, a la niebla que dificulta visibilidad de los agentes y a cierta laxitud de las autoridade